In [9]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [38]:
data = pd.read_csv('../Downloads/archive/fashion-mnist_train.csv')

In [39]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
data = np.array(data)
m,n = data.shape


x_train = data[0][1:n]
y_train = data[0][0]

In [22]:
def download_minist_dataset():
    # downloads and unzips the mnist dataset to the computer
    
    import os
    import urllib.request
    import urllib 
    from zipfile import ZipFile
    URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
    FILE = 'fashion_mnist_images.zip'
    FOLDER = 'fashion_mnist_images'
    if not os.path.isfile(FILE):
        print(f'Downloading {URL} and saving as {FILE}...')
        urllib.request.urlretrieve(URL, FILE)
    print('Unzipping images...')
    with ZipFile(FILE) as zip_images:
        zip_images.extractall(FOLDER)
    print('Done!')

def load_mnist_dataset(dataset, path):
    # loads and preprocesses the mnist dataset

    import cv2 # to install: pip install opencv-python
    import os
    # Scan all the directories and create a list of labels
    labels = os.listdir(os.path.join(path, dataset))
    # Create lists for samples and labels
    X = []
    y = []
    # For each label folder
    for label in labels:
        # And for each image in given folder
        for file in os.listdir(os.path.join(path, dataset, label)):
            # Read the image
            image = cv2.imread(os.path.join(
                path, dataset, label, file), cv2.IMREAD_UNCHANGED)
            # And append it and a label to the lists
            X.append(image)
            y.append(label)
    # Convert the data to proper numpy arrays and return
    return np.array(X), np.array(y).astype('uint8')

def create_data_mnist(path):
    # Load both sets separately
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    # And return all the data
    return X, y, X_test, y_test

download_minist_dataset()
X, y, X_test, y_test = create_data_mnist('fashion_mnist_images')

Unzipping images...
Done!


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,  25,  50,   0,   0,   2,   0,   1,   2,   3,
          3,   0],
       [  

In [54]:
def init_params():
    # generating random values for each element of the array of the first layer
    w1 = np.random.rand(784, 10)
    b1 = np.random.rand(1, 10)
    
    # generating random values for the hidden layer of 10 neurons
    w2 = np.random.rand(10, 10)
    b2 = np.random.rand(1, 10)
    
    return w1, b1, w2, b2

# activation that returns only positive values
def reLU(layer):
    return np.maximum(0, layer)

# activation that returns a number between 0 and 1
def softmax(layer):
    return np.exp(layer) / np.sum(np.exp(layer))

# taking weights and biases plus the layer x to obtain 
def forward_prop(w1, b1, w2, b2, X):
    z1 = np.dot(X, w1) + b1
    # np.dot(x, w1)
    a1 = reLU(z1)
    z2 = np.dot(a1, w2) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2

# what the fuck is this doing? Creating a 0s matrix and substituting with 1 for some reason?
#def one_hot(y):
 #   one_hot_y = np.zeros((y.size, y.max() + 1))
 #   one_hot_y[np.arange(y.size), y] = 1
 #   one_hot_y = one_hot_y.T
    
# derivative of reLU activation function
def d_reLU(layer):
    return layer > 0
    
# 
def back_prop(z1, a1, z2, a2, w1, w2, x, y):
    d_z2 = a2 - y
    d_w2 = 1 / m * d_z2.dot(a1.T)
    d_b2 = 1 / m * np.sum(d_z2)
    d_z1 = w2.T.dot(d_z2) * d_reLU(z1)
    d_w1 = 1 / m * d_z1.dot(x.T)
    d_b1 = 1 / m * np.sum(d_z1)
    return d_w1, d_b1, d_w2, d_b2

def update_params(w1, b1, w2, b2, d_w1, d_b1, d_w2, d_b2, alpha):
    w1 = w1 - alpha * d_w1
    b1 = b1 - alpha * d_b1
    w2 = w2 - alpha * d_w2
    b2 = b2 - alpha * d_b2
    return w1, b1, w2, b2

In [55]:
def get_predictions(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y)/y.size

def grad_desc(x, y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
        d_w1, d_b1, d_w2, d_b2 = back_prop(z1, a1, z2, a2, w1, w2, x, y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, d_w1, d_w2, d_b1, d_b2, alpha)
        if i % 50 == 0:
            print('Iteration ', i)
            print('Accuracy ', get_accuracy(get_predictions(a2), y))
    return w1, b1, w2, b2

In [56]:
w1, b1, w2, b2 = grad_desc(x_train, y_train, 500, 0.1)

<ipython-input-54-bd04f9dea434>:18: RuntimeWarning: overflow encountered in exp
  return np.exp(layer) / np.sum(np.exp(layer))
<ipython-input-54-bd04f9dea434>:18: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(layer) / np.sum(np.exp(layer))


ValueError: shapes (10,10) and (1,10) not aligned: 10 (dim 1) != 1 (dim 0)